## Вариант 3. Контроль количества параметров

**Цель:** Создание компактной сети.

- Создайте архитектуру, используя **не более 50 000 параметров**.
- **Условие:** Один слой должен быть **1x1 сверткой**, чтобы уменьшить число каналов.
- **Эксперимент:** Подсчитайте количество параметров каждого слоя и убедитесь, что общая сумма не превышает лимита.

In [1]:
import matplotlib.pyplot as plt
import numpy as np

import torch


from torchvision import datasets, transforms